In [0]:
%pip install -r ../requirements.txt


*** WARNING: max output size exceeded, skipping output. ***

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 688.0/688.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.9/798.9 kB 5.5 MB/s eta 0:00:00

In [0]:

%pip install mlflow
%pip install -qU databricks-langchain

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%restart_python

In [0]:
import asyncio
from uuid import uuid4

from langchain_core.messages import AIMessage, HumanMessage

# Before importing the project files, make sure the src directory is in the Python path.
# This is usually handled by installing the package in editable mode (`uv pip install -e .`)
# as recommended in the README.
from open_deep_research.deep_researcher import deep_researcher
from open_deep_research.state import AgentState

# For rich output in notebooks that can render HTML/Markdown
try:
    from IPython.display import display, Markdown
    IS_NOTEBOOK = True
except ImportError:
    IS_NOTEBOOK = False


async def main():
    """
    An example of how to run the Open Deep Research agent interactively,
    handling the clarification step. This is useful for environments like
    Databricks notebooks or local terminals.
    """
    # 1. Set up your environment
    # Make sure you have OPENAI_API_KEY, TAVILY_API_KEY, etc., set in your environment.
    # You can copy .env.example to .env and fill in your keys.

    # 2. Define the initial input for the research agent
    research_topic = input("Please enter the research topic: ")
    messages = [HumanMessage(content=research_topic)]

    # 3. Configure the agent
    # A unique thread_id is important for keeping conversations separate.
    config = {
        "configurable": {
            "thread_id": str(uuid4()),
        }
    }

    # 4. Start the interactive loop
    while True:
        if IS_NOTEBOOK:
            display(Markdown("--- \n*Running Research Agent...*\n"))
        else:
            print("\n--- Running Research Agent ---\n")

        # The agent is a LangGraph object, which you can invoke.
        # We pass the current message history to the agent.
        final_state: AgentState = await deep_researcher.ainvoke(
            {"messages": messages}, config
        )

        # The state contains the full message history. We'll update our local
        # list with the messages from the agent's run.
        messages = final_state["messages"]

        # 5. Check for a final report
        final_report = final_state.get("final_report")
        if final_report:
            if IS_NOTEBOOK:
                display(Markdown("# Research Complete"))
                display(Markdown(final_report))
            else:
                print("\n--- Research Complete ---\n")
                print(final_report)
            break

        # 6. If no report, the agent has likely asked a clarifying question.
        last_message = messages[-1]
        if isinstance(last_message, AIMessage):
            if IS_NOTEBOOK:
                display(Markdown(last_message.content))
            else:
                print(f"\nAI: {last_message.content}")

            # Prompt the user for their response to the AI's question
            # In a notebook, input() renders as a textbox.
            user_response = input("\nYour response: ")
            messages.append(HumanMessage(content=user_response))

# In Databricks notebooks, use 'await main()' instead of 'asyncio.run(main())'
await main()

Please enter the research topic:  future of AI healthcare

--- 
*Running Research Agent...*


I'd be happy to research the future of AI in healthcare for you. To provide the most relevant and comprehensive report, could you please clarify a few details:

**Scope & Focus:**
- Are you interested in specific areas of AI healthcare (e.g., diagnostics, drug discovery, personalized medicine, robotic surgery, telemedicine)?
- Do you want to focus on particular timeframes (e.g., next 5 years, next decade, long-term projections)?

**Geographic Focus:**
- Should this cover global trends or focus on specific regions/countries?

**Perspective:**
- Are you looking at this from a particular angle (e.g., business opportunities, clinical applications, regulatory challenges, patient outcomes)?

**Report Format:**
- What level of detail would you prefer (executive summary, comprehensive analysis, technical deep-dive)?
- Are there specific sections you'd like included (market projections, key players, challenges, opportunities)?


Your response:  1. drug discovery 2. global trends 3. clinical applications 4. technical deep-dive, challenges

--- 
*Running Research Agent...*


/Workspace/Users/jason.yip@tredence.com/odr/open_deep_research/src/open_deep_research/deep_researcher.py:298: RuntimeWarning: coroutine 'Pregel.ainvoke' was never awaited
  HumanMessage(content=tool_call["args"]["research_topic"])


# Research Complete

# The Future of AI in Drug Discovery: A Comprehensive Technical Analysis

## Current State and Emerging AI Technologies in Drug Discovery

Artificial intelligence has fundamentally transformed the pharmaceutical industry's approach to drug discovery, introducing unprecedented capabilities across the entire drug development pipeline. The current landscape showcases sophisticated AI applications that are revolutionizing traditional methodologies and accelerating the path from laboratory to clinic.

### Target Identification and Validation

Modern AI systems leverage advanced machine learning algorithms to identify novel therapeutic targets with remarkable precision. Deep learning models analyze vast genomic datasets, protein structures, and disease pathways to predict previously unknown drug targets. Graph neural networks have emerged as particularly powerful tools for understanding complex biological networks and protein-protein interactions. These systems can process multi-omics data, including genomics, proteomics, and metabolomics, to identify targets that traditional methods might overlook.

Natural language processing (NLP) technologies are simultaneously mining decades of scientific literature to extract hidden relationships between genes, proteins, and diseases. These AI systems can identify potential targets by analyzing patterns across millions of research papers, patents, and clinical reports, effectively serving as intelligent research assistants that never tire of reading scientific literature.

### Lead Compound Discovery and Optimization

Generative AI models have revolutionized the process of molecular design and lead optimization. Variational autoencoders (VAEs), generative adversarial networks (GANs), and transformer-based models can now generate novel molecular structures with desired properties. These systems learn from vast chemical databases to understand the relationship between molecular structure and biological activity, enabling the design of compounds with optimized pharmacokinetic and pharmacodynamic properties.

Reinforcement learning algorithms are being employed to navigate the vast chemical space more efficiently than traditional high-throughput screening methods. These systems can explore millions of potential compounds virtually, identifying promising candidates for synthesis and testing. Advanced molecular dynamics simulations powered by AI can predict drug-target interactions with increasing accuracy, reducing the need for extensive laboratory testing in early stages.

### Clinical Trial Design and Patient Stratification

AI technologies are transforming clinical trial design through sophisticated patient stratification algorithms and predictive modeling. Machine learning models analyze electronic health records, genetic profiles, and biomarker data to identify optimal patient populations for specific trials. These systems can predict patient responses to treatments, enabling more precise inclusion and exclusion criteria that improve trial success rates.

Digital twin technologies are emerging as powerful tools for simulating clinical trials before they begin. These AI-powered models can predict trial outcomes, optimize dosing regimens, and identify potential safety concerns before human testing commences. Natural language processing systems are also streamlining the regulatory submission process by automatically generating and reviewing clinical trial documentation.

### Regulatory Approval Pathways

AI is increasingly being integrated into regulatory processes, with agencies worldwide developing frameworks for evaluating AI-driven drug discovery. Machine learning models are being used to predict regulatory approval likelihood based on preclinical and clinical data patterns. These systems analyze historical approval data to identify factors that contribute to successful regulatory submissions.

Automated pharmacovigilance systems powered by AI are enhancing post-market surveillance capabilities. These systems can detect adverse events and safety signals from diverse data sources, including social media, electronic health records, and spontaneous reporting systems, providing real-time safety monitoring capabilities that surpass traditional methods.

## Global Trends and Regional Developments

The adoption of AI in drug discovery varies significantly across global markets, reflecting differences in regulatory environments, investment patterns, and technological infrastructure. Understanding these regional variations is crucial for comprehending the future trajectory of AI-driven pharmaceutical innovation.

### North American Market Leadership

North America, particularly the United States, continues to lead in AI drug discovery innovation, driven by substantial venture capital investment and a supportive regulatory environment. The FDA has established clear pathways for AI-driven drug development, including guidance documents for machine learning-based medical devices and drug development tools. Major pharmaceutical companies in the region have formed extensive partnerships with AI startups, creating a robust ecosystem for innovation.

The National Institutes of Health (NIH) has launched several initiatives supporting AI in drug discovery, including the Accelerating Medicines Partnership (AMP) program, which leverages AI to identify new therapeutic targets for complex diseases. Academic institutions across the United States are establishing dedicated AI drug discovery centers, creating a pipeline of trained researchers and novel technologies.

### European Integration and Collaboration

Europe has adopted a more collaborative approach to AI drug discovery, with the European Medicines Agency (EMA) developing harmonized guidelines for AI applications across member states. The European Union's Horizon Europe program has allocated significant funding for AI-driven pharmaceutical research, emphasizing cross-border collaboration and data sharing initiatives.

The European approach emphasizes ethical AI development and data privacy, with GDPR

[Trace(trace_id=tr-16c3f82eda6cb415e6d41ce60007b51d), Trace(trace_id=tr-bf5335fb365cacc322de80f5531f42ed)]

In [0]:
import asyncio
from uuid import uuid4

from langchain_core.messages import AIMessage, HumanMessage

from open_deep_research.deep_researcher import deep_researcher
from open_deep_research.state import AgentState
from open_deep_research.utils import get_all_tools
from open_deep_research.configuration import MCPConfig, MCPServerConfig

try:
    from IPython.display import display, Markdown
    IS_NOTEBOOK = True
except ImportError:
    IS_NOTEBOOK = False


async def test_multi_mcp():
    """
    An example of how to run the Open Deep Research agent with multiple MCP servers.
    """
    # 1. Define the MCP server configurations
    mcp_servers = [
        MCPServerConfig(
            url="https://adb-8333330282859393.13.azuredatabricks.net/api/2.0/mcp/genie/01f08636c357189e81d584d6bbfef815",
            auth_required=True,
        ),
        MCPServerConfig(
            url="https://this-is-a-dummy-url.com/api/2.0/mcp/genie/dummy-mcp-server",
            auth_required=False,
        ),
        MCPServerConfig(
            url="https://another-dummy-url.com/api/2.0/mcp/genie/another-dummy-mcp-server",
            auth_required=True,
            token="directly-provided-token"
        )
    ]
    
    # 2. Define the initial input for the research agent
    research_topic = "test"
    messages = [HumanMessage(content=research_topic)]

    # 3. Configure the agent
    config = {
        "configurable": {
            "thread_id": str(uuid4()),
            "mcp_config": MCPConfig(
                servers=mcp_servers,
                tools=["all"],
            )
        }
    }

    # 4. Get all available tools
    all_tools = await get_all_tools(config)
    
    # 5. Print the names of all the tools
    if IS_NOTEBOOK:
        display(Markdown("## Available Tools:"))
        for tool in all_tools:
            tool_name = tool.name if hasattr(tool, "name") else tool.get("name", "web_search")
            display(Markdown(f"- {tool_name}"))
    else:
        print("Available Tools:")
        for tool in all_tools:
            tool_name = tool.name if hasattr(tool, "name") else tool.get("name", "web_search")
            print(f"- {tool_name}")

await test_multi_mcp()
